## Defining and Calling Functions

1.  Work through the explanation in the slides (and chapter)
    of how to define a function in our little language
    and then how to call one.
2.  Look at the documentation for Python's `ChainMap` class
    and modify the implementation from the slides to use that
    to manage environments.

## 1. Add to Implementation from Slides

I added a function `run_setf` which sets a function to a variable. the difference between this and the `set` function is that it doens't evaluate the value until the function is called.

I added a function `run_evalf` to evaluate a function.

Right now, my implementation is only able to define functions with no parameters.

In [7]:
def run_add(env, args):
    assert len(args) == 2
    left = run(env, args[0])
    right = run(env, args[1])
    # let python do the actual addition
    # our langauge builds upon python
    return left + right

def run_mul(env, args):
    assert len(args) == 2
    left = run(env, args[0])
    right = run(env, args[1])
    return left * right

def run_get(env, args):
    assert len(args) == 1
    name = args[0]
    assert name in env, f"Unkown variable {name}"
    return env[name]

def run_seq(env, args):
    assert len(args) > 0
    result = None
    for expr in args:
        result = run(env, expr)
    return result

def run_set(env, args):
    assert len(args) == 2
    name = args[0]
    value = run(env, args[1])
    env[name] = value
    return value

def run_setf(env, args):
    assert len(args) == 2
    name = args[0]
    value = args[1] # don't evaluate when setting function
    env[name] = value
    return value

# LAZY EVALUATION
# true case won't error when cond is false
def run_if(env, args):
    assert len(args) == 3
    cond = args[0]
    if_true = args[1]
    if_false = args[2]
    # again, rely on python (i.e. level below us)
    if run(env, cond):
        return run(env, if_true)
    else:
        return run(env, if_false)
    
# EAGER EVALUATION
# true case will error even if cond is false
# this is what R does UGH
# def run_if(env, args):
#     assert len(args) == 3
#     cond = run(env, args[0])
#     if_true = run(env, args[1])
#     if_false = run(env, args[0])
#     # again, rely on python (i.e. level below us)
#     if cond:
#         return if_true
#     else:
#         return if_false


def run_evalf(env, args):
    assert len(args) == 1
    fun_name = args[0]
    fun = env[fun_name]
    return run(env, fun)

def run(env, expr):
    if not isinstance(expr, list):
        return expr
    op = expr[0]
    args = expr[1:]
    assert op in OPS, f"unkown operation {op}"
    func = OPS[op]
    return func(env, args)

OPS = {
    "add": run_add,
    "mul": run_mul,
    "get": run_get,
    "seq": run_seq,
    "set": run_set,
    "if": run_if,
    "evalf": run_evalf,
    'setf': run_setf
}

In [8]:
program = [
    'seq',
    ['setf', 'my_func', ['add',2,3]],
    ['evalf', 'my_func']
]
print(run(env = {}, expr = program))

5


is equivalent to

In [9]:
def my_fun():
    return 2 + 3

my_fun()

5

## 2. Use Python's ChainMap class to manage environments

https://realpython.com/python-chainmap/

ChainMap is a "handy tool for managing multiple scopes and contexts". From what I can tell, a ChainMap behaves just like a dictionary for accessing, setting, and checking contents.

I replaced the environment with a ChainMap, and everything seems to behave the same. I think the main benefit of using a chain map is to easily use multiple environments and still interact with it as if it's a single dictionary (for instance: `ChainMap({'test_var': 'a'}, {'test_2': 'b'})`).

In [10]:
from collections import ChainMap

c = ChainMap({'test_var': 'a'})

In [11]:
c['test_var']

'a'

In [12]:
'test_var' in c

True

In [13]:
c['abs'] = 2
c

ChainMap({'test_var': 'a', 'abs': 2})

In [17]:
c = ChainMap({'test_var': 'a'}, {'test_2': 'b'})
c['test_var'], c['test_2']

('a', 'b')

In [22]:
def run_add(env, args):
    assert len(args) == 2
    left = run(env, args[0])
    right = run(env, args[1])
    # let python do the actual addition
    # our langauge builds upon python
    return left + right

def run_mul(env, args):
    assert len(args) == 2
    left = run(env, args[0])
    right = run(env, args[1])
    return left * right

def run_get(env, args):
    assert len(args) == 1
    name = args[0]
    assert name in env, f"Unkown variable {name}"
    return env[name]

def run_seq(env, args):
    assert len(args) > 0
    result = None
    for expr in args:
        result = run(env, expr)
    return result

def run_set(env, args):
    assert len(args) == 2
    name = args[0]
    value = run(env, args[1])
    env[name] = value
    return value

# LAZY EVALUATION
# true case won't error when cond is false
def run_if(env, args):
    assert len(args) == 3
    cond = args[0]
    if_true = args[1]
    if_false = args[2]
    # again, rely on python (i.e. level below us)
    if run(env, cond):
        return run(env, if_true)
    else:
        return run(env, if_false)
    
# EAGER EVALUATION
# true case will error even if cond is false
# this is what R does UGH
# def run_if(env, args):
#     assert len(args) == 3
#     cond = run(env, args[0])
#     if_true = run(env, args[1])
#     if_false = run(env, args[0])
#     # again, rely on python (i.e. level below us)
#     if cond:
#         return if_true
#     else:
#         return if_false


def run_evalf(env, args):
    assert len(args) == 1
    fun_name = args[0]
    fun = env[fun_name]
    return run(env, fun)

def run(env, expr):
    if not isinstance(expr, list):
        return expr
    op = expr[0]
    args = expr[1:]
    assert op in OPS, f"unkown operation {op}"
    func = OPS[op]
    return func(env, args)

OPS = {
    "add": run_add,
    "mul": run_mul,
    "get": run_get,
    "seq": run_seq,
    "set": run_set,
    "if": run_if,
    "evalf": run_evalf
}

variables = {
    "firas": 1,
    "jenna": 3
}
chainmap_env = ChainMap(variables)

program = [
    'seq',
    ['set', 'my_func', ['add',['get','jenna'],3]],
    ['evalf', 'my_func']
]
print(run(env = chainmap_env, expr = program))

6


## Arrays

Implement fixed-size one-dimensional arrays:

1.  `["array", 10]` creates an array of 10 elements.
    (If you want to assign it to a variable you could use `["set", "var", ["array", 10]]`.)
2.  Other instructions that you design get and set array elements by index.

In [19]:
def run_array(env, args):
    assert len(args) == 1
    array_len = args[0]
    return [None]*array_len

def run_set_element(env, args):
    assert len(args) == 3
    array = run(env, args[0])
    idx = run(env, args[1])
    value = run(env, args[2])
    array[idx] = value
    return value

def run_get_element(env, args):
    assert len(args) == 2
    array = run(env, args[0])
    idx = run(env, args[1])
    return array[idx]

OPS = {
    "add": run_add,
    "mul": run_mul,
    "get": run_get,
    "seq": run_seq,
    "set": run_set,
    "if": run_if,
    "setf": run_setf,
    "evalf": run_evalf,
    "array": run_array,
    'get_element': run_get_element,
    'set_element': run_set_element
}

variables = {
    "firas": 1,
    "jenna": 3
}
chainmap_env = ChainMap(variables)

program = [
    'seq',
    ['set', 'my_array', ['array', 10]],
    ['set_element', ['get', 'my_array'], 4, 'abc'],
    ['get_element', ['get', 'my_array'], 4]
]
print(run(env = chainmap_env, expr = program))

abc


## While Loops

Implement a `while` loop instruction. Your implementation can use either a Python `while` loop or recursion.

I also implemented a "less than" operator where `['lt',left,right]` checks whether `left < right`. I implement a while loop that takes in a `condition` and a `do` operator. I test this out where the condition checks whether a variable is less than 3, and for each iteration increases the value of that variable.

In [44]:
def run_while(env, args):
    condition = args[0]
    do = args[1]
    it = 1
    while run(env, condition):
        run(env, do)
        it += 1
    return(it)

def run_lt(env, args):
    left = run(env, args[0])
    right = run(env, args[1])
    return left < right

OPS = {
    "add": run_add,
    "mul": run_mul,
    "get": run_get,
    "seq": run_seq,
    "set": run_set,
    "if": run_if,
    "evalf": run_evalf,
    "setf": run_setf,
    "array": run_array,
    'get_element': get_element,
    'set_element': set_element,
    'while': run_while,
    'lt': run_lt
}

In [45]:
variables = {
    "firas": 1,
    "jenna": 2
}

chainmap_env = ChainMap(variables)

run(env = chainmap_env, expr = ["if", ['lt', ['get', 'jenna'], 3], True, False])

True

In [49]:
variables = {
    "firas": 1,
    "jenna": 0
}
chainmap_env = ChainMap(variables)

print(run(env = chainmap_env, expr = ['get','jenna']))

program = [
    'while', 
    ["if", ['lt', ['get', 'jenna'], 3], True, False], 
    ['set', 'jenna', ['add', ['get', 'jenna'], 1]]
]
print(run(env = chainmap_env, expr = program))

print(run(env = chainmap_env, expr = ['get','jenna']))

0
4
3


Same as

In [50]:
jenna = 0
it = 1
while jenna < 3:
    jenna += 1
    it += 1
print(it)

4
